# MCP (3 Agent system to parse any local repository and explain the code)


## Author: Dhruv Saxena (Cambridge)

In [6]:
from pathlib import Path

from mcp import ClientSession, StdioServerParameters
from mcp.client.sse import sse_client
from mcp.client.stdio import stdio_client

from autogen import LLMConfig
from autogen.agentchat import AssistantAgent
from autogen.mcp import create_toolkit
import json
import anyio
import asyncio



# Only needed for Jupyter notebooks
import nest_asyncio
nest_asyncio.apply()

from autogen.agentchat.group import (
    AgentNameTarget,
    AgentTarget,
    AskUserTarget,
    ContextExpression,
    ContextStr,
    ContextStrLLMCondition,
    ContextVariables,
    ExpressionAvailableCondition,
    ExpressionContextCondition,
    GroupChatConfig,
    GroupChatTarget,
    Handoffs,
    NestedChatTarget,
    OnCondition,
    OnContextCondition,
    ReplyResult,
    RevertToUserTarget,
    SpeakerSelectionResult,
    StayTarget,
    StringAvailableCondition,
    StringContextCondition,
    StringLLMCondition,
    TerminateTarget,
)

from autogen import (
    ConversableAgent,
    GroupChat,
    GroupChatManager,
    LLMConfig,
    UserProxyAgent,
    register_function,
)

from autogen.agentchat.group.patterns import (
    DefaultPattern,
    ManualPattern,
    AutoPattern,
    RandomPattern,
    RoundRobinPattern,
)


from autogen import ConversableAgent, UpdateSystemMessage
from autogen.agents.experimental import DocAgent
import os
import copy
from typing import Any, Dict, List
from pydantic import BaseModel, Field


from autogen.agentchat import initiate_group_chat, a_initiate_group_chat

In [7]:
mcp_server_path = Path("mcp_filesystem.py")

In [ ]:
joker_message = """
You are a code exploration assistant.

Your tasks:
1. Recursively list all files using the `list_all_files_recursive` tool.
2. Read source files (.py, .js, .cpp, .md, etc.) using the `read_file` tool only when needed.
3. Answer user questions about how the code works, starting from high-level and drilling into details only when necessary.
4. Identify the entry point and key modules, summarize file purposes, and show how components interact.
5. Think step-by-step — you can ask yourself questions and call tools accordingly before replying.
6. After completing your summary, ask the user what they would like to explore next.
""".strip()




from pydantic import BaseModel,Field




default_llm_config = {'cache_seed': 42,
                     'temperature': 1.,
                     'top_p': 0.05,
                     'config_list': [{'model': 'gpt-4o',
                                      'api_key': "", #ADD YOUR API KEY
                                      'api_type': 'openai'}],
                     'timeout': 1200}

joker_config_list = copy.deepcopy(default_llm_config)
# joker_config_list['config_list'][0]['response_format'] = JokeResponse


joker =  ConversableAgent(
    name="joker",
    system_message=r"""
        Your job is to summarize the codebase in your folder recursively:
        1. Use `list_all_files_recursive` to get all code files.
        2. Use `read_file` on each relevant file (e.g., .py, .js, .cpp).
        3. Summarize the purpose of each file.
        4. Identify how they connect and find the entry point.
        5. After you're done summarizing, say "The file has been read."

        After the summary, ask the user if they want to:
        - Drill down into a specific file
        - Understand a function or module
        - Or ask another question about the codebase.
        """,
    llm_config = joker_config_list,
    update_agent_state_before_reply=[UpdateSystemMessage(joker_message),],
)



task = """
Recursively traverse the user given directory and summarize what the code inside the directory does.
"""

initial_agent = joker

In [9]:
import asyncio
from autogen import UserProxyAgent

class AsyncUserProxyAgent(UserProxyAgent):
    async def a_get_human_input(self, prompt: str) -> str:
        print(prompt)
        loop = asyncio.get_event_loop()
        return await loop.run_in_executor(None, input, "> ")


In [ ]:
async def create_toolkit_and_run(session: ClientSession) -> None:
    # Create toolkit with tools
    toolkit = await create_toolkit(session=session)

    # Define agents
    mcp_agent = ConversableAgent(
        name="mcp_agent",
        system_message=r"Read the file in your folder and skip non-code files.",
        llm_config=default_llm_config,
    )

    joker = ConversableAgent(
        name="joker",
        system_message=r"You are Joker. Help the user understand the summarized codebase.",
        llm_config=default_llm_config,
    )

    user_proxy = AsyncUserProxyAgent(name="real_user")


    # Register tools with MCP
    toolkit.register_for_llm(mcp_agent)
    toolkit.register_for_execution(mcp_agent)

    # Handoff chain: mcp → joker → user → joker → user ...
    mcp_agent.handoffs.set_after_work(AgentTarget(joker))
    joker.handoffs.set_after_work(AgentTarget(user_proxy))
    user_proxy.handoffs.set_after_work(AgentTarget(joker))


    # Optional LLM condition to reinforce flow
    mcp_agent.handoffs.add_llm_conditions([
        OnCondition(
            condition=StringLLMCondition(prompt="The files have been read."),
            target=AgentTarget(joker),
        ),
    ])

    # Ask user for directory path
    user_input = input("Enter directory path to traverse and summarize: ")

    # Define task prompt
    task_prompt = (
        f"List and briefly summarize the purpose of each file in this directory: {user_input}. "
        "Ignore executable files and directories. Don't read *.out files. "
        "Use `list_all_files_recursive` to get all files. Use `read_file` to read each. "
        "Skip files that are not code, too large, or unreadable."
    )

    # Define agent interaction pattern
    agent_pattern = DefaultPattern(
        agents=[mcp_agent, joker, user_proxy],
        initial_agent=mcp_agent,
    )

    # Run the task
    hello = await a_initiate_group_chat(
        pattern=agent_pattern,
        messages=[task_prompt],
        max_rounds=20,  # Increase if needed
    )
    

# Server params for MCP stdio agent
server_params = StdioServerParameters(
    command="python",
    args=[str(mcp_server_path), "stdio"],
)

import os
os.environ["AUTOGEN_USE_DOCKER"] = "False"

# Launch chat
async with stdio_client(server_params) as (read, write), ClientSession(read, write) as session:
    await session.initialize()
    await create_toolkit_and_run(session)